# Different ML classifiers: an overview

Following is an overview of some of the popular machine learnign methods (classical methods only here, no deep neural networks).

## Decision Trees

A decision tree is a binary tree.  At each of the internal nodes, it chooses a feature $i$ and a threshold $t$.  Each leaf has a value.  Evaluation of the model is just traversal of the tree from the root.  At each node, for example $j$, we go down the left branch if $X_{ji} \le t$ and the right branch otherwise.  The value of the model $f(X_{ji})$ is the value at the value at the terminating leaf of this traveral.  Below, we show a picture of this on small decision tree trained on the iris data set.  Notice that each internal node has a decision criterion and each leaf has the breakdown of label classes left at this leaf of the tree.  


### Random Forests

A random forest is just an ensemble of decision trees.  The predicted value is just the average of the trees (for both regression and classification problems - for classification problems, it is the probabilities that are averaged).  You can adjust `n_estimators` to change the number of trees in the forest.  If each tree is trained on the same subset of data, why aren't they identical?  Two reasons:
1. **Subsampling**: each tree is actually trained on a random selected (with replacement) subset (i.e. bootstrap)
1. **Maximum Features**: the optimal split comes from a randomly selected subset of the features.  In scikit-learn, this feature is controlled by `max_features`.

### Random Forest Training Algorithm and Tuning Parameters

A Random Forest is pretty straightforward to train once you know how a Decision Tree works.  In fact, their construction can even be parallelized.  

Below, various parameters that affect decision tree and random forest training are discussed. 

The first two parameters we mention are the most important, and tuning them can often improve performance:

**numTrees**: Number of trees in the forest.

Increasing the number of trees will decrease the variance in predictions, improving the model’s test-time accuracy.
Training time increases roughly linearly in the number of trees.


**maxDepth**: Maximum depth of each tree in the forest.
Increasing the depth makes the model more expressive and powerful. However, deep trees take longer to train and are also more prone to overfitting.
In general, it is acceptable to train deeper trees when using random forests than when using a single decision tree. One tree is more likely to overfit than a random forest (because of the variance reduction from averaging multiple trees in the forest).

The next two parameters generally do not require tuning. However, they can be tuned to speed up training.

**subsamplingRate**: This parameter specifies the size of the dataset used for training each tree in the forest, as a fraction of the size of the original dataset. The default (1.0) is recommended, but decreasing this fraction can speed up training.

**featureSubsetStrategy**: Number of features to use as candidates for splitting at each tree node. The number is specified as a fraction or function of the total number of features. Decreasing this number will speed up training, but can sometimes impact performance if too low.

## Linear SVM

The canonical Support Vector Machine is the linear one.  Assume we have two groups labeled by $y = \pm 1$.  Then we are trying to find the line $\beta$ such that $X \beta + \beta_0$ maximially separates the points in our two classes:

If the two classes can be separated by a linear hyperplane (picture on the left), we want to maximize the **margin** $M$ of the **boundary region**.  A little bit of math can show us that finding the largest separation is actually solved by the minimization problem

$$
\min_{\beta, \beta_0} \|\beta\| \\
\mbox{subject to } y_j (X_{j\cdot} \cdot \beta + \beta_0) \ge 1 \quad \mbox{for } j = 1,\ldots,N
$$

The picture and the equation are equivalent: in the picture we are setting the margin to be $M$ and finding the largest margin possible.  In the equation, we are setting the margin to be $1$ and finding the smallest $\beta$ that will make that true.  So $\beta$ and $M$ are related through $\| \beta \| = \frac{1}{M}$.  If the two classes cannot be separated (picture on the right), we will have to add a forgiveness terms $\xi$,

$$
\min_{\beta, \beta_0} \|\beta\| \\
\mbox{subject to } \left\{ \begin{array} {cl} 
 y_j (X_{j\cdot} \cdot \beta + \beta_0) \ge (1-\xi_j) & \mbox{for } j = 1,\ldots,N \\
 \xi_j \ge 0 & \mbox{for } j = 1,\ldots,N \\
 \sum_j \xi_j \le C
\end{array}\right.
$$

for some constant $C$.  The constant $C$ is an important tradeoff.  It corresponds to the total "forgiveness budget" (see the last constraint).  The larger $C$, the forgiveness we have and the wider the margin $M$ can be.  We can rewrite the constrained optimization problem as the primal Lagrangian function with Lagrange multipliers $\alpha_j \ge 0$, $\mu_j \ge 0$, and $\gamma \ge 0$,  for each of our three constraints:

$$ L_P(\gamma) = \min_{\beta, \beta_0, \xi} \max_{\alpha, \mu} \frac{1}{2} \| \beta \|^2 - \sum_j \alpha_j \left[y_j (X_{j \cdot} \cdot \beta + \beta_0 - (1-\xi_j)\right] - \sum_j \mu_j \xi_j  + \gamma \sum_j \xi_j$$

There is a one-to-one correspondence between $\gamma$ and $C$.  By taking first order conditions, first-order conditions, the dual Lagrangian problem can be formulated as

$$
L_D(\gamma) = \max_{\alpha} \sum_j \alpha_j - \frac{1}{2} \sum_{j, j'} \alpha_j \alpha_{j'} y_j y_{j'} X_{j \cdot} \cdot X_{j' \cdot} \,. \\
\mbox{subject to } \left\{ \begin{array} {cl} 
0 = \sum_j \alpha_j y_j \\
0 \le \alpha_j \le \gamma & \mbox{for } j = 1,\ldots,N
\end{array}\right.
$$

This is now a reasonably straightforward quadratic programming problem.  It is solved via [Sequential Minimization Optimization](https://en.wikipedia.org/wiki/Sequential_minimal_optimization).  Once we have solved this problem for $\alpha$, we can easily work out the coefficients from

$$ \beta = \sum_j \alpha_j y_j X_{j \cdot} $$

**Key takeaways**:
1. Critically, only points inside the margin or on the wrong side of the margin ($j$ for which $\xi_j > 0$) affect the SVM (see the picture).  This is intuitively clear from the picture.  In the dual form, this is because $\alpha_j$ is the Lagrangian constraint corresponding to $y_j (X_{j\cdot} \cdot \beta + \beta_0) \ge (1-\xi_j)$ and Complementary Slackness shows tells us that $\alpha_j > 0$ is non-zero only when the constraint is binding ($y_j (X_{j\cdot} \cdot \beta + \beta_0) = (1-\xi_j)$), i.e. we're in the boundary region.  This is meaning the **Support Vector** in "SVM": only the vectors in the boundary-the **Support Vectors**-contribute to the solution.
1. $C$ or $\gamma$ give a trade-off between the amount of forgiveness and the size of the margin or boundary region.  Hence, it controls how many points affect the SVM (based on the distance from the boundary).

Below, we plot out a simple two-class linear SVM on some synthetic data


## Non-linear SVM

What if we don't believe that our data can be cleanly split by a linear hyperplane?  The common way to incorporate non-linear features is to have a non-linear function $h(X_{j\cdot})$ (possibly to a higher-dimensional feature space with dimension $p'$ where $p' \ge p$) and to train on that space.  One intuition is that there's a higher-dimensional space in which the data is has a linear separation and $h$ gives a non-linear mapping into that space.

### Kernel Trick

The **Kernel Trick** in SVM tells us that rather than directly computing the (potentially very large) vectors $h(X_{j \cdot})$, we can just modify the Kernel.  If we use the transformed data $h(X_{j \cdot})$, the dual Lagrangian would be

$$ \max_{\alpha} \sum_j \alpha_j - \frac{1}{2} \sum_j \sum_{j'} \alpha_j \alpha_{j'} y_j y_{j'} h(X_{j \cdot}) \cdot h(X_{j' \cdot}) $$

We can rewrite

$$h(X_{j \cdot}) \cdot h(X_{j' \cdot})  = K(X_{j \cdot}, X_{j' \cdot})$$ 

for some non-linear Kernel $K$.  Our problem then becomes,

$$ \max_{\alpha} \sum_j \alpha_j - \frac{1}{2} \sum_j \sum_{j'} \alpha_j \alpha_{j'} y_j y_{j'} K(X_{j \cdot}, X_{j' \cdot}) $$

There's a one-to-one correspondence between Kernel functions and functions $h$ (although $h$'s range may be infinite dimensional).  Some common Kernels include

<table>
<tr>
<th>Kernel</th>
<th>$K(x,x')$</th>
<th>Scikit `kernel` parameter</th>
</tr>

<tr>
<td>Linear Kernel</td>
<td>$x \cdot x'$</td>
<td>`kernel='linear'`</td>
</tr>

<tr>
<td>$d$-th Degree Polynomial</td>
<td>$(r + c x \cdot x')^d$</td>
<td>`kernel='poly'`</td>
</tr>

<tr>
<td>Radial Kernel</td>
<td>$ \exp(- c \|x - x' \|^2) $</td>
<td>`kernel='rbf'`</td>
</tr>

<tr>
<td>Neural Network Kernel</td>
<td>$\tanh(c x \cdot x' + r)$</td>
<td>`kernel='sigmoid'`</td>
</tr>
</table>

The benefit of using a Kernel is that we don't have to compute a very high-dimensional (possibly infinite-dimensional) $h$.  All that complexity is just wrapped into the kernel $K$.